# cu_inj Live

In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

In [ ]:
%pylab inline

In [ ]:
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from lcls_live.tools import isotime

import os

In [ ]:
BEAMPATH = 'cu_spec'

## Datamaps, and all PVs needed

In [ ]:
DM = get_datamaps(BEAMPATH)

# Hack in 10 Hz
df = DM['bpms'].data
DM['bpms'].data['pvname'] = [name[:-2]+'TH' for name in df['pvname']]

DM.keys()

In [ ]:
# datamaps to exclude
DENYLIST = ['correctors']

In [ ]:
# PVs needed
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
len(PVLIST)

## Archiver restore

In [ ]:
ISOTIME = '2021-04-21T08:10:25.000000-07:00'

In [ ]:
# Optional off-site setup

# Open an SSH tunnel in a terminal like:
# ssh -D 8080 <some user>@<some SLAC machine>

OFFSITE=True

if OFFSITE:
    os.environ['http_proxy']='socks5h://localhost:8080'
    os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
    os.environ['ALL_PROXY']='socks5h://localhost:8080'

In [ ]:
PVDATA = lcls_archiver_restore(PVLIST, ISOTIME)

## EPICS

In [ ]:
import epics
from time import sleep, time

In [ ]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST}
sleep(5)

In [ ]:
def get_pvdata():
        
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    
    #print(f'Acquired settings from EPICS at: {itime}')
    
    return pvdata
PVDATA = get_pvdata()

# Tao conveniences

In [ ]:
def tao_commands(pvdata):
    cmds = []
    for name, dm in DM.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

In [ ]:
def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        for cmd in CMDS:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

In [ ]:
def toggle_beam():
    tao.cmd('set global track_type = beam') 
    tao.cmd('set global track_type = single') 

## Form commands using PVDATA and datamaps

In [ ]:
tao_commands(PVDATA)[0:5]

# Start Tao

In [ ]:
from pytao import Tao

In [ ]:
MODEL_ROOT = f'$LCLS_LATTICE/bmad/models/{BEAMPATH}/'
init = f'-init {MODEL_ROOT}/tao.init'
tao = Tao(init)
init

In [ ]:
# Turn on the beam
tao.cmd('set beam_init n_particle = 2000')
toggle_beam()

In [ ]:
%%tao
place floor none
place top2 bpm_orbit
place middle2 bunch_sigma_xy 

sc top2 -.2 .2

set graph middle2 y%label = "\gs\fn\dx\u, \gs\fn\dy\u (\gmm)"
set curve middle2.g.x y_axis_scale_factor = 1e6
set curve middle2.g.y y_axis_scale_factor = 1e6
sc middle2 0 600


x-s all 0 15

In [ ]:
#tao.cmd(f'call {MODEL_ROOT}/scripts/SC.tao')

In [ ]:
while True:
    #sleep(.001)
    t1 = time()
    pvdata = get_pvdata()
    cmds = tao_commands(pvdata)
    tao.cmd('set global plot_on = F;set global lattice_calc_on = F')
    tao.cmds(cmds); # Apply
    
    # Chaos monkey to see something
    #xx = 2*np.random.rand()-1
    #tao.cmd(f'set ele qE01 k1 = 4*{xx}')      
    
    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('set global plot_on = T')
    toggle_beam()
    
    dt = time()-t1
    #print(dt)


In [ ]:
%%time
#toggle on and off
tao.cmd('set global track_type = beam', raises=False) 
tao.cmd('set global track_type = single') 

In [ ]:
%%tao
place top2 beambeta
place middle2 bunch_sigma_xy
x-a all s
sc

## Get particles 

In [ ]:
from pmd_beamphysics import ParticleGroup

In [ ]:
P = ParticleGroup(data=tao.bunch_data('OTR2'))
# Select out live particles
P = P[P.status==1]

In [ ]:
P.plot('t', 'energy')

In [ ]:
P

In [ ]:
# Make a function for this
def get_beam(ele):
    P = ParticleGroup(data=tao.bunch_data(ele))
    # Select out live particles
    P = P[P.status==1]    
    
    return P

In [ ]:
P1 = get_beam('OTR2')
P1.plot('delta_t', 'delta_energy')

In [ ]:
P2 = get_beam('OTR2')
P2.plot('x', 'y')

# Get giant table of bunch stats

In [ ]:
import pandas as pd

In [ ]:
# Get list of indices
IX = tao.lat_list('*', 'ele.ix_ele')

# Stop before the dump
S_STOP = tao.ele_head('OTR2')['s']

stats = []
for ele in IX:
    d = tao.bunch_params(ele)
    # Skipl ones with no beam
    if d['charge_live'] == 0:
        continue
    
    if d['s'] > S_STOP:
        break
        
    stats.append(d)

df = pd.DataFrame(stats)#.set_index('ix_ele')
df

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plt.plot(df['s'], df['sigma_x']*1e6, label=r'$\sigma_x$')
plt.plot(df['s'], df['sigma_y']*1e6, label=r'$\sigma_y$')
plt.xlabel('s (m)')
plt.ylabel('beam sizes (µm)')
plt.legend();

In [ ]:
# Get an array of where the beam is saved
s_saved = np.array(df[df['beam_saved']]['s'])

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
plt.plot(df['s'], df['norm_emit_x']*1e6, label=r'$\epsilon_{n,x}$')
plt.plot(df['s'], df['norm_emit_y']*1e6, label=r'$\epsilon_{n,y}$')
plt.xlabel('s (m)')
plt.ylabel('Beam emittance(mm-mrad)')


# This is where the beam is saved
plt.scatter(s_saved, np.ones(len(s_saved))*0, color = 'black', marker = 'x')


plt.legend();